# 01 - Dati e profili social

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from io import StringIO
import re
import numpy as np
import pandas as pd
from requests import get
from pandas import read_csv
from unidecode import unidecode
from bs4 import BeautifulSoup as BS

Ho scaricato le 12 tabelle dei giocatori più preziosi da Transfermarkt.

https://www.transfermarkt.it/spieler-statistik/wertvollstespieler/marktwertetop?land_id=0&ausrichtung=alle&spielerposition_id=alle&altersklasse=alle&jahrgang=0&plus=1

Siccome non è possibile fare un ciclo sull'indirizzo (vengono mostrati 25 risultati per pagina, quindi in totale sono 12 tabelle) ho scaricato e salvato in un file HTML le 12 tabelle così da poter rendere replicabile la prima parte del notebook (infatti i seguaci Instagram, Facebook, etc sono in **continuo** - praticamente istantaneo - cambiamento).

In [2]:
# leggo il file html e lo converto in stringa con la funzione .read()
f = open('tables/top300_values_14122018.html', 'r', encoding="utf8").read()

# tratto la stringa html esattamente come un sito qualunque con BeautifulSoup
html_tables = BS(f, 'lxml')

In [3]:
# estraggo dalle tabelle il maggior numero di informazioni possibile

id_player = [x['id'] for x in html_tables.find_all('a', attrs={'class': 'spielprofil_tooltip'})]
nomi = [x.text for x in html_tables.find_all('a', attrs={'class': 'spielprofil_tooltip'})]
eta = [x.find_all('td')[5].text for x in html_tables.find_all('tr', attrs={'class': 'odd'})]
nazioni = [x.find_all('td')[6].find('img')['title'] for x in html_tables.find_all('tr', attrs={'class': 'odd'})]
squadre = [x.find_all('td')[7].find('img')['alt'] for x in html_tables.find_all('tr', attrs={'class': 'odd'})]
ruoli2 = [x.find_all('tr')[1].text for x in html_tables.find_all('table', attrs={'class': 'inline-table'})]
valori = [int(x.find('b').text.split(',')[0])*1000000 for x in html_tables.find_all('td', attrs={'class': 'rechts'})]

Voglio tenere sia il ruolo in generale, sia il ruolo nel dettaglio:

In [4]:
attaccante = ['Ala destra', 'Ala sinistra', 'Punta', 'Seconda punta']
centrocampista = ['Centrale', 'Centrocampista', 'Mediano', 'Trequartista']
difensore = ['Difensore', 'Terzino']

ruoli = ['Portiere']*len(ruoli2)

k = 0
for x in ruoli2:
    for r in attaccante:
        if r in x:
            ruoli[k] = 'Attaccante'
    for r in centrocampista:
        if r in x:
            ruoli[k] = 'Centrocampista'
    for r in difensore:
        if r in x:
            ruoli[k] = 'Difensore'
    k+=1
set(ruoli)

{'Attaccante', 'Centrocampista', 'Difensore', 'Portiere'}

In [5]:
df_dict = {
    'id':id_player,
    'nome':nomi,
    'eta':eta,
    'nazione':nazioni,
    'ruolo':ruoli,
    'ruolo_detail':ruoli2,
    'squadra':squadre,
    'valore':valori,
}

df = pd.DataFrame(df_dict)
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000


## Social Data (followers, likes, etc.)

Carico il dataframe con tutte le URLs dei tre social in questione

In [6]:
table_url = pd.read_csv('tables/Top300_Urls - Sheet1.csv')
table_url.head()

,name,transfermarkt,instagram,facebook,twitter
0,Kylian Mbappé,https://www.transfermarkt.it/kylian-mbappe/pro...,https://www.instagram.com/k.mbappe/,https://www.facebook.com/kylianmbappeofficiel/,https://twitter.com/KMbappe
1,Neymar,https://www.transfermarkt.it/neymar/profil/spi...,https://www.instagram.com/neymarjr,https://www.facebook.com/neymarjr,https://twitter.com/neymarjr
2,Lionel Messi,https://www.transfermarkt.it/lionel-messi/prof...,https://www.instagram.com/leomessi,https://www.facebook.com/LeoMessi,NaN
3,Mohamed Salah,https://www.transfermarkt.it/mohamed-salah/pro...,https://www.instagram.com/mosalah/,https://www.facebook.com/momosalah,https://twitter.com/MoSalah
4,Harry Kane,https://www.transfermarkt.it/harry-kane/profil...,https://www.instagram.com/harrykane/,https://www.facebook.com/harrykaneofficial,https://twitter.com/HKane


### Funzione che esegue scraping sui profili social

In [7]:
# getFollowers() prende in input una serie di link e il nome del social network
# restituisce come output una lista di interi pari al numero di followers/likes

def getFollowers(pdSeries,socialName,Output=False):
    if Output:
        print('1'+'-'*98+'100%'); 
    k = 1
    
    hs = {"User-Agent":"Mozilla/5.0"}
    followers = []
    errors = []
    
    # ciclo for all'interno della serie, u = url
    
    for i,u in pdSeries.iteritems():
        
        # se u=url non è nan (float):
         
        if not isinstance(u, float):
            
            # siccome temo che a volte requests non funzioni,
            # gli dico di provare n volte a scaricare la pagina
            # controllo che l'output sia un intero;
            # finché resta stringa vuol dire che qualcosa è andato storto
            
            s = ''   # per il controllo che sia un intero
            n = 1    # per fare massimo n prove
            
            
            while(isinstance(s, str) & (n < 4)):                
                try:
                    page = requests.get(u).text
                    
                    # scraping da INSTAGRAM
                    if socialName == 'Instagram':
                        s = int(re.findall(r'"edge_followed_by":{"count":+(\d*)', page)[0])
                        
                    # scraping da FACEBOOK
                    elif socialName == 'Facebook':
                        #s = int(re.findall(r'<div>Piace\sa\s+([\d|.]*)', page)[0].replace('.', ''))
                        s = int(re.findall(r'(\d*)\s+personnes+\s+aiment+\s+ça',re.sub('\xa0','',page))[0])
                    # scraping da TWITTER
                    elif socialName == 'Twitter':
                        a = BS(page,'lxml').find('a',attrs={'class':'ProfileNav-stat','data-nav':'followers'})['title']
                        s = int(a.split(' ')[0].replace('.',''))
                except: 
                    s = ''
                    n += 1
            
            # se anche dopo 3 volte non è riuscito ad estrarre il numero di followers
            # salvo dentro errori l'url problematica
            # e appendo a followers un errore
            
            
            if (isinstance(s, str)):
                errors.append(u)
                followers.append('errore')
                
            if len(errors) > 30:
                break
            
            # altrimenti è tutto ok e appendo il numero di followers
            
            else:
                followers.append(s)
        
        # l'url può essere np.nan (cioè float). in tal caso appendi np.nan
        
        else:
            followers.append(np.nan)
              
        if (k%3==0):
            if Output:
                print('=', end='')
        k += 1
    
    # prima printo gli errori e poi
    # ritorno la lista di followers
    print('\n');  
    
    if Output:
        print(errors)
    
    return(followers)

### *Instagram*

Spiegazione di cosa fa la formula quando `socialName='Instagram'`.

Per estrarre il numero esatto di seguaci, utilizzo una **regular expression** (pacchetto `re`):

`re.findall(r'"edge_followed_by":{"count":+(\d*)', page)[0]`

In sostanza noto che nel codice HTML di ogni profilo (contenuto in `page`) il numero di seguaci è preceduto dalla stringa: `"edge_followed_by":{"count":`

Quindi chiedo di estrarre - indicando tra parentesi tonde `( )` - tutti i numeri in sequenza - indicato dal `\d*` - specificando appunto la stringa che precede.

`findall` restituisce una lista, quindi prendo l'elemento `[0]`.

Questa è una via alternativa - **molto** rapida ed efficace - all'uso di BeautifulSoup.

In [8]:
df['instagram_followers'] =  getFollowers(table_url.instagram, 'Instagram', Output=True)

1--------------------------------------------------------------------------------------------------100%

[]


In [10]:
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000,24019703.0
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000,107716768.0
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000,103764191.0
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000,21661975.0
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000,7389999.0


In [11]:
df[df.instagram_followers.isnull()]

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers
31,65230,Sergio Busquets,30,Spagna,Centrocampista,Mediano,FC Barcellona,80000000,NaN
116,357565,Rodri,22,Spagna,Centrocampista,Mediano,Atlético de Madrid,40000000,NaN
220,50174,Danny Rose,28,Inghilterra,Difensore,Terzino sinistro,Tottenham Hotspur,30000000,NaN
260,229005,Abdou Diallo,22,Francia,Difensore,Difensore centrale,Borussia Dortmund,25000000,NaN


In totale ci sono 4 NaN: il numero è corretto.

### *Facebook*

Utilizzo la stessa tecnica che è stata usata per Instagram: una regular expression.

`re.findall(r'<div>Piace\sa\s+([\d|.]*)', page)[0]`

In questo caso il numero di likes è sempre preceduto da un `<div>` e da "Piace a ". 

Sfrutto questa particolarità per estrarre i valori, che vengono poi trasformati in intero dopo aver rimosso i punti dalle stringhe.

In [13]:
df['facebook_followers'] =  getFollowers(table_url.facebook, 'Facebook', Output=True)

1--------------------------------------------------------------------------------------------------100%

[]


In [14]:
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers,facebook_followers
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000,24019703.0,1639467.0
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000,107716768.0,60699722.0
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000,103764191.0,89898268.0
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000,21661975.0,11016135.0
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000,7389999.0,1660408.0


### *Twitter*

In questo caso, per estrarre i followers, ho utilizzato BeautifulSoup, in quanto risulta il modo più agevole (seppur più lento).

In [17]:
df['twitter_followers'] =  getFollowers(table_url.twitter, 'Twitter', Output=True)

1--------------------------------------------------------------------------------------------------100%

[]


In [18]:
df.head()

,id,nome,eta,nazione,ruolo,ruolo_detail,squadra,valore,instagram_followers,facebook_followers,twitter_followers
0,342229,Kylian Mbappé,19,Francia,Attaccante,Ala destra,FC Paris Saint-Germain,180000000,24019703.0,1639467.0,2252200.0
1,68290,Neymar,26,Brasile,Attaccante,Ala sinistra,FC Paris Saint-Germain,180000000,107716768.0,60699722.0,41636656.0
2,28003,Lionel Messi,31,Argentina,Attaccante,Ala destra,FC Barcellona,180000000,103764191.0,89898268.0,NaN
3,148455,Mohamed Salah,26,Egitto,Attaccante,Ala destra,FC Liverpool,150000000,21661975.0,11016135.0,7457524.0
4,132098,Harry Kane,25,Inghilterra,Attaccante,Punta centrale,Tottenham Hotspur,150000000,7389999.0,1660408.0,2453183.0


In [19]:
df.to_csv('columns/df1.csv', index=False, encoding='ansi')

# 02 - Nazione delle squadre

In [4]:
df1 = read_csv('tables/nome_squadra.csv')
df1.head()

,nome,squadra
0,Kylian Mbappé,FC Paris Saint-Germain
1,Neymar,FC Paris Saint-Germain
2,Lionel Messi,FC Barcellona
3,Mohamed Salah,FC Liverpool
4,Harry Kane,Tottenham Hotspur


In [6]:
df2 = read_csv('tables/squadra_nazione.csv', encoding='ansi')
df2.head()

,Squadra,Nazione
0,FC Barcellona,Spagna
1,Real Madrid CF,Spagna
2,Atlético de Madrid,Spagna
3,Valencia CF,Spagna
4,FC Siviglia,Spagna


In [9]:
df2.index = df2.Squadra
df2.drop('Squadra',axis=1,inplace=True)
df2.head()

,Nazione
Squadra,
FC Barcellona,Spagna
Real Madrid CF,Spagna
Atlético de Madrid,Spagna
Valencia CF,Spagna
FC Siviglia,Spagna


In [10]:
df3 = df1.join(df2, on='squadra')
df3.head()

,nome,squadra,Nazione
0,Kylian Mbappé,FC Paris Saint-Germain,Francia
1,Neymar,FC Paris Saint-Germain,Francia
2,Lionel Messi,FC Barcellona,Spagna
3,Mohamed Salah,FC Liverpool,Inghilterra
4,Harry Kane,Tottenham Hotspur,Inghilterra


In [11]:
df3.to_csv('columns/nazione_squadra.csv', index=False, encoding='ansi')

# 03 - Google Hits

Quante volte un giocatore appare facendo una ricerca Google?

In [19]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
from numpy import log
from io import StringIO
import re
from unidecode import unidecode

In [4]:
url = 'https://docs.google.com/spreadsheets/d/1RPiYmNxy8NWKp-uu_swOkRkTuq9cqJNnzJpl5BPliDM/export?format=csv&id=1RPiYmNxy8NWKp-uu_swOkRkTuq9cqJNnzJpl5BPliDM&gid=2001946666'
r = requests.get(url)
data = r.content.decode('utf-8')
df = pd.read_csv(StringIO(data))
df = pd.DataFrame(df[['nome_chiave']])
df['nome_url'] = [x.replace(" ","+").replace("-","+").lower() for x in df.nome_chiave]

Funzione per scaricare hit di Google 

In [13]:
s1 = 'https://www.google.com/search?q='
lista_1 = [s1 + x for x in df.nome_url]
df['google_hits'] = np.nan

In [14]:
for i in range(300):
    link = lista_1[i]
    if isinstance(df.google_hits[i], float):
        soup = BS(requests.get(link).text, 'lxml')
        result = soup.find('div', attrs = {'id': 'resultStats'}).text.strip()
        df.loc[i, 'google_hits'] = result

In [23]:
l = [''.join(re.findall('(\d)',unidecode(x))) for x in df.google_hits]
df['google_hits1'] = l
df[['google_hits1']].to_csv('columns/google_hits.csv', index=False)

# 04 - Instagram Media Likes

Average of latest 12 posts likes

In [3]:
df = read_csv('tables/nome_instagram.csv', encoding='ansi')
df.instagram[df.instagram.isnull()]=''
df['urls'] = ['https://web.stagram.com/'+x.strip('/').split('/')[-1] for x in df.instagram.values]
df['likes_avg'] = np.nan
df.loc[df.instagram=='', 'urls'] = np.nan

In [26]:
driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()
i = 0
for url in df.urls:
    if df.loc[i,'likes_avg'] > 0:
        i+=1
        pass
    else:
        if isinstance(url,float):
            i+=1
            pass
        else:
            try:  
                driver.get(url)
                sleep(3)
                divs = [x.text for x in driver.find_elements_by_class_name('card-footer')]
                if divs:
                    total_likes = []
                    for div in divs:
                        total_likes.append(int(div.split(' ')[0].replace(',','')))
                    likes_avg = np.array(total_likes).mean().round(1)
                    df.loc[i,'likes_avg'] = likes_avg      
            except:
                pass
            i += 1
driver.quit()
df.to_csv('columns/insta_likes_avg.csv', encoding='ansi')

# 05 - Instagram Hashtags

Quante foto hanno come **#hashtag** il nome di un calciatore?

In [3]:
def f(x):
    return unidecode(x).replace(' ', '').replace('-','').replace("'", '').lower()

dfNames = pd.read_csv('tables/nome_nomechiave.csv')
dfNames['nome2'] = dfNames['nome_chiave'].apply(f)
dfNames['hashtag_tot'] = np.nan

In [ ]:
i = 0
for n in dfNames.nome2:
    if dfNames.hashtag_tot[i] > 0:
        i += 1
    else:
        try:
            url = 'https://www.instagram.com/explore/tags/'+ n +'/'
            page = get(url).text
            dfNames.iloc[i,-1] = int(re.findall('"edge_hashtag_to_media":{"count":+(\d+)', page)[0])
            i += 1
            sleep(3)
        except:
            i += 1
#     print(i-1,dfNames.nome[i-1])

dfNames.to_csv('columns/insta_hashtags.csv', encoding='ansi', index=False)

# 06 - Statistiche Wikipedia

### Scraping Wikipedia Stats

In [ ]:
df = read_csv('tables/Top300_Urls - Sheet1.csv')
df = df[['nome','wiki']]

In [ ]:
s1 = 'https://tools.wmflabs.org/pageviews/?project=en.wikipedia.org&platform=all-access&agent=user&start=2018-01-01&end=2018-12-20&pages='
df['urls'] = [s1 + x.split('/')[-1] for x in df.wiki]
df['wiki_visual'] = np.nan
df['wiki_avg'] = np.nan

In [ ]:
driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

#=======================

i = 0
for url in df.urls:
    if isinstance(df.wiki_visual[i], float):
        driver.get(url)
        sleep(3)       
        try:
            stats = driver.find_elements_by_class_name('linear-legend--counts')
            d = dict([tuple(x.text.split(':')) for x in stats if x.text])
            df.loc[i, 'wiki_visual'] = d['Visualizzazioni per pagina'].strip().replace(".","")
            df.loc[i, 'wiki_avg'] = d['Media giornaliera'].strip().replace(".","")
            #print(i, end=' ')
            i += 1
        except:
            i += 1
    else:
        i += 1

#=======================

driver.quit()

df.to_csv('columns/wiki_visual_and_avg.csv',index=False,encoding='ansi')

# 07 - Kicker Data

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='ansi')
df['nome_chiave_link'] = [x.replace(' ', '+').replace('-','+').lower() for x in df['nome_chiave']]
s1 = 'http://www.kicker.de/home/24-09-1970/24-12-2018/0-1100-31000-37000-36000-39000-33000-22000-508024-1200-1000-/25/0/0/suchergebnisseartikel.html?suchbegriff='
s2 = 'http://www.kicker.de/home/01-01-2018/24-12-2018/0-1100-31000-37000-36000-39000-33000-22000-508024-1200-1000-/25/0/0/suchergebnisseartikel.html?suchbegriff='
df['urls_all'] = [s1+x for x in df.nome_chiave_link]
df['urls_18'] = [s2+x for x in df.nome_chiave_link]
df['kicker_all'] = np.nan
df['kicker_18'] = np.nan
df = df[['nome', 'urls_all', 'urls_18', 'kicker_all', 'kicker_18']]

In [ ]:
for i in range(300):
    url1 = df.urls_all[i]
    url2 = df.urls_18[i]
    
    if isinstance(df.kicker_google_all[i], float):
        page = get(url1).text
        result1 = BS(page, 'lxml').find('div', attrs={'class': 'suchdachzeile'}).text
        df.loc[i,'kicker_all'] = result1
    else:
        pass
    
    if isinstance(df.kicker_google_18[i], float):
        page = get(url2).text
        result2 = BS(page, 'lxml').find('div', attrs={'class': 'suchdachzeile'}).text
        df.loc[i,'kicker_18'] = result2
    else:
        pass
    
    #print(i, end=' ')

In [9]:
def f(x):
    return x.split(' ')[0]

df['kicker_all'] = df['kicker_all'].apply(f) 

df['kicker_18'] = df['kicker_18'].apply(f) 

df.iloc[:,-2:].to_csv('columns/kicker_data.csv', index=False)

# 08 - Corriere dello Sport

## Articoli di qualunque data

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='ansi')
df['nome_chiave_link'] = [x.replace(' ', '+').replace('-','+').lower() for x in df['nome_chiave']]
s1 = 'http://www.corrieredellosport.it/search?key=('
df['urls_all'] = [s1+x+')' for x in df.nome_chiave_link]
df['corriere_all'] = np.nan
df = df[['nome', 'urls_all', 'corriere_all']]

In [22]:
for i in range(300):
    url = df.urls_all[i]
    
    # solito trucco per riempire man mano la colonna
    # così se da errore non perdo nulla
    # e posso proseguire riavviando la cella:
    
    if isinstance(df.corriere_all[i], float):
        
        # uso espressione regex (pacchetto "re")
        # per non passare da Beautiful Soup
        
        l = re.findall('>+(\d+)\s+articoli</a>', get(url).text)
        
        # if lista: <--- questa espressione dà True
        # se la lista ha almeno un elemento dentro di essa.
        # serve per evitare errori banali tipo lista[0]
        # ma la lista è vuota quindi [0] non esiste --> errore
        
        if l:
            df.loc[i, 'corriere_all'] = l[0]

## Articoli del 2018

In [79]:
df['corriere_18'] = np.nan

Funzione che prende in input una pagina e restituisce il numero di volte che appare 2018 nei link agli articoli di quella pagina:

In [80]:
def sum18(url):
    p = BS(get(url).text,'lxml')
    y = p.select('div.main-section__main article.article div.article__content a')
    return sum(['2018' in x['href'] for x in y])

In [ ]:
for i in range(300):
    s = np.nan
    
    # solito trucco per riempire man mano la colonna
    # così se da errore non perdo nulla
    # e posso proseguire riavviando la cella:
    
    if isinstance(df.corriere_18[i], float):
        
        url = df.urls_all[i]+'%20AND%20(type%3Aarticolo)'
        s = sum18(url)
        N = int(p.select('span.page')[0].text.split(' ')[-1])
        
        # ciclo for nelle pagine alla ricerca delle date con 2018 nei link
        # in ogni pagina ci sono 10 link i quali possono contenere o meno 2018
        # se nessun link contiene 2018 vuol dire che posso uscire dal ciclo
        # infatti pare che siano ordinati cronologicamente (grazie a dio)
        # sommo ad ogni passo il numero di volte che il link contiene 2018
        # cumulo le somme, e se la somma è 0 allora break
        
        for j in range(2,N+1):
            
            url2 = url+'&p='+str(j)
            s2 = sum18(url2) 
            s += s2
            
            if s2 == 0:
                break
        
        # salvo come stringa per assicurarmi che non venga visto come numeric
        
        df.loc[i, 'corriere_18'] = str(s)

df[['corriere_all', 'corriere_18']].to_csv('columns/corriere_dello_sport.csv')

# 09 - Skysport, Sky Italia e Eurosport

## Skysport.com

### Qualunque anno

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:skysports.com&source=lnms&tbm=nws'
df['urls_all'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['skysport_google_all'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.skysport_google_all[i], float):
        url = df.urls_all[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'skysport_google_all'] = result
        #print((i, df.nome[i]), end=' ')

### 2018

In [ ]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:skysports.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_18'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['skysport_google_18'] = np.nan

In [ ]:
driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.skysport_google_18[i], float):
        url = df.url_18[i]
        driver.get(url)
        sleep(np.random.uniform(2))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(2))
        result = driver.find_element_by_id('resultStats').text
        df.loc[i, 'skysport_google_18'] = result
        #print((i,df.nome[i]), end=' ')
    
driver.quit()

## Sky Sport Italia

### Qualunque anno

In [ ]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:sport.sky.it&source=lnms&tbm=nws'
df['urls_all'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['skyitalia_google_all'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.skyitalia_google_all[i], float):
        url = df.urls_all[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'skyitalia_google_all'] = result
        #print((i, df.nome[i]), end=' ')

### 2018

In [ ]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:sport.sky.it&tbm=nws&source=lnt&tbs=qdr:y'
df['url_18'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['skyitalia_google_18'] = np.nan

In [ ]:
driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(290,300):
    if isinstance(df.skyitalia_google_18[i], float):
        url = df.url_18[i]
        driver.get(url)
        sleep(np.random.uniform(2))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(2))
        result = driver.find_element_by_id('resultStats').text
        df.loc[i, 'skyitalia_google_18'] = result
        #print((i,df.nome[i]), end=' ')
    
driver.quit()

## Eurosport

### Qualunque anno

In [144]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:eurosport.com&source=lnms&tbm=nws'
df['urls_all'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['eurosport_google_all'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.eurosport_google_all[i], float):
        url = df.urls_all[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'eurosport_google_all'] = result
        #print((i, df.nome[i]), end=' ')

### 2018

In [ ]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:eurosport.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_18'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['eurosport_google_18'] = np.nan

In [ ]:
driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.eurosport_google_18[i], float):
        url = df.url_18[i]
        driver.get(url)
        sleep(np.random.uniform(2))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(2))
        result = driver.find_element_by_id('resultStats').text
        df.loc[i, 'eurosport_google_18'] = result
        #print((i,df.nome[i]), end=' ')
    
driver.quit()

### Pulizia dati

In [ ]:
c = ['skysport_google_all', 'skysport_google_18', 
     'skyitalia_google_all', 'skyitalia_google_18', 
     'eurosport_google_all', 'eurosport_google_18']
df2 = df[c]

In [ ]:
for col in df2.columns:
    df2.loc[df2[col].isnull(), col] = 'Circa 0 risultati'

def f(x):
    return x.replace('.', '')

for j in range(6):
    df2.iloc[:, j] = df2.iloc[:, j].apply(f)

def f2(x):
    return re.findall('\d+',x)[0]

for nomecol in df2.columns:
    df2[nomecol] = df2[nomecol].apply(f2)
    
df2.to_csv('columns/skysports_skyitalia_eurosport.csv')

# 10 - The Guardian

## Qualunque anno

In [2]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.theguardian.com&source=lnms&tbm=nws'
df['urls_all'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['guardian_google_all'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.guardian_google_all[i], float):
        url = df.urls_all[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'guardian_google_all'] = result
        #print((i, df.nome[i]), end=' ')

## 2018

In [7]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.theguardian.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_18'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['guardian_google_18'] = np.nan

In [ ]:
driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.guardian_google_18[i], float):
        url = df.url_18[i]
        driver.get(url)
        sleep(np.random.uniform(2))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(2))
        result = driver.find_element_by_id('resultStats').text
        df.loc[i, 'guardian_google_18'] = result
        #print((i,df.nome[i]), end=' ')
    
driver.quit()

## Pulizia dati

In [50]:
df[['guardian_google_all', 'guardian_google_18']].head()

,guardian_google_all,guardian_google_18
0,Circa 639 risultati,"Circa 354 risultati (0,22 secondi)"
1,Circa 3.300 risultati,"Circa 687 risultati (0,16 secondi)"
2,Circa 5.860 risultati,"Circa 964 risultati (0,19 secondi)"
3,Circa 1.960 risultati,"Circa 852 risultati (0,20 secondi)"
4,Circa 4.420 risultati,"Circa 1.000 risultati (0,21 secondi)"


In [ ]:
def f(x):
    x=x.replace('.','')
    x=re.findall('(\d+)',x)
    if x:
        return x[0]

df2 = df[['guardian_google_all', 'guardian_google_18']]

df2.loc[0:300, 'guardian_google_all'] = df2.guardian_google_all.apply(f)
df2.loc[0:300, 'guardian_google_18'] = df2.guardian_google_18.apply(f)

df2.to_csv('columns/guardian_data.csv')

# 11 - Gazzetta dello Sport

## Qualunque anno

In [2]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.gazzetta.it&source=lnms&tbm=nws'
df['urls_all'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['gs_google_all'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.gs_google_all[i], float):
        url = df.urls_all[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'gs_google_all'] = result
        print((i, df.nome[i]), end=' ')

## 2018

In [2]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.gazzetta.it&tbm=nws&source=lnt&tbs=qdr:y'
df['url_18'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['gs_google_18'] = np.nan

# ------

driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.gs_google_18[i], float):
        url = df.url_18[i]
        driver.get(url)
        sleep(np.random.uniform(1))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(1))
        result = driver.find_element_by_id('resultStats').text
        df.loc[i, 'gs_google_18'] = result
        print((i,df.nome[i]), end=' ')
    
driver.quit()

### Pulizia dati

In [16]:
def f(x):
    x = x.replace(',', '')
    x = re.findall('(\d+)', x)
    if x:
        return x[0]
df2 = df.gs_google_all
pd.DataFrame(df2.apply(f)).to_csv('columns/gazzetta_google_all.csv', index=False)

In [34]:
def f(x):
    x = x.replace('.', '')
    x = re.findall('(\d+)', x)
    if x:
        return x[0]
df2 = df.gs_google_18
pd.DataFrame(df2.apply(f)).to_csv('columns/gazzetta_google_18.csv', index=False)

# 12 - France Football e AS

## France Football

### Qualunque anno

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:francefootball.com&source=lnms&tbm=nws'
df['url_ff'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]

headers = {"User-Agent":"Mozilla/5.0"}
results_ff = []

# si estrae il numero di risultati di google in corrispondenza della ricerca di ciascun giocatore sulla rivista FranceFootball
for link in df.url_ff:
    soup = BS(get(link).text,'lxml')
    stringa = soup.find('div', attrs = {'id': 'resultStats'}).text.strip().replace('.','')
    results_ff.append(int(str([int(s) for s in stringa if s.isdigit()]).strip('[]').replace(',','').replace(' ','')))

# si aggiorna il dataset df con la nuova colonna
df['ff_google_all'] = results_ff

### 2018

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:francefootball.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_ff'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['ff_google_18'] = np.nan

# ------

driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.ff_google_18[i], float):
        url = df.url_ff[i]
        driver.get(url)
        sleep(np.random.uniform(1))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(1))
        result = driver.find_element_by_id('resultStats').text.strip().replace('.','')
        result = int(str([int(s) for s in result if s.isdigit()]).strip('[]').replace(',','').replace(' ',''))
        df.loc[i, 'ff_google_18'] = result
        print((i,df.nome[i]), end=' ')
    
driver.quit()

## AS.com

### Qualunque anno

In [ ]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:as.com&source=lnms&tbm=nws'
df['url_as'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]

headers = {"User-Agent":"Mozilla/5.0"}
results_as = []

# si estrae il numero di risultati di google in corrispondenza della ricerca di ciascun giocatore sulla rivista AS
for link in df.url_as:
    soup = BS(get(link).text,'lxml')
    results_as.append(soup.find('div', attrs = {'id': 'resultStats'}).text.strip().split(' ')[1].replace('.',''))

# si aggiorna il dataset df con la nuova colonna
df['as_google_all'] = results_as

### 2018

In [ ]:
df = read_csv('tables/nome_nomechiave.csv', encoding='utf8')
s1 = 'https://www.google.com/search?q='
s2 = '+site:as.com&tbm=nws&source=lnt&tbs=qdr:y'
df['url_as'] = [s1 + x.replace(' ', '+').lower() + s2 for x in df.nome_chiave]
df['as_google_18'] = np.nan

# ------

driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.as_google_18[i], float):
        url = df.url_as[i]
        driver.get(url)
        sleep(np.random.uniform(1))
        el = driver.find_element_by_link_text('Strumenti')
        webdriver.ActionChains(driver).move_to_element(el).click(el).perform()
        sleep(np.random.uniform(1))
        result = driver.find_element_by_id('resultStats').text.strip().replace('.','')
        result = int(str([int(s) for s in result if s.isdigit()]).strip('[]').replace(',','').replace(' ',''))
        df.loc[i, 'as_google_18'] = result
        print((i,df.nome[i]), end=' ')
    
driver.quit()

In [ ]:
d = df[['ff_google_all', 'ff_google_18', 'as_google_all', 'as_google_18']]
d.to_csv('columns/ff_as_google.csv', index=False)

# 13 - Presenze, Goal e Assist

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re
import numpy as np
import pandas as pd
from requests import get
from pandas import read_csv
from unidecode import unidecode
from bs4 import BeautifulSoup as BS

In [45]:
df_urls = read_csv('tables/Top300_Urls - Sheet1.csv')
df_ruolo = read_csv('tables/ruolo.csv')
names = [x.split('/')[3] for x in df_urls.transfermarkt]
ids = [x.split('/')[6] for x in df_urls.transfermarkt]
ruolo = df_ruolo.ruolo

In [50]:
df = pd.DataFrame({
    'pres_2018_tot':[np.nan]*300,
    'goal_2018_tot':[np.nan]*300,
    'assist_2018_tot':[np.nan]*300,
    'pres_2018_CL':[np.nan]*300,
    'goal_2018_CL':[np.nan]*300,
    'assist_2018_CL':[np.nan]*300,
    'pres_2017_tot':[np.nan]*300,
    'goal_2017_tot':[np.nan]*300,
    'assist_2017_tot':[np.nan]*300,
    'pres_2017_CL':[np.nan]*300,
    'goal_2017_CL':[np.nan]*300,
    'assist_2017_CL':[np.nan]*300,
    'pres_tot':[np.nan]*300,
    'goal_tot':[np.nan]*300,
    'assist_tot':[np.nan]*300,
    'pres_tot_CL':[np.nan]*300,
    'goal_tot_CL':[np.nan]*300,
    'assist_tot_CL':[np.nan]*300,
})

In [65]:
for i in range(300):
    url18 = 'https://www.transfermarkt.it/'+names[i]+'/leistungsdaten/spieler/'+ids[i]+'/saison/2018/plus/1#gesamt'
    url17 = 'https://www.transfermarkt.it/'+names[i]+'/leistungsdaten/spieler/'+ids[i]+'/saison/2017/plus/1#gesamt'
    urltot = 'https://www.transfermarkt.it/'+names[i]+'/leistungsdaten/spieler/'+ids[i]+'/plus/1?saison=ges'
    
    if isinstance(df.assist_tot_CL[i], float):
        
        page18 = BS(get(url18, headers = {"User-Agent":"Mozilla/5.0"}).text, 'lxml')
        page17 = BS(get(url17, headers = {"User-Agent":"Mozilla/5.0"}).text, 'lxml')
        pagetot = BS(get(urltot, headers = {"User-Agent":"Mozilla/5.0"}).text, 'lxml')


        # all 2018 =============================================================
        try:
            trs = page18.select('div#yw2 tfoot')[0].find_all('tr')        
            for tr in trs:
                tds = tr.find_all('td')
                pres = tds[2].text.strip()
                gol = tds[3].text.strip()
                assist = tds[4].text.strip()
            df.loc[i, 'pres_2018_tot'] = pres
            df.loc[i, 'goal_2018_tot'] = gol
            df.loc[i, 'assist_2018_tot'] = assist
        except:
            df.loc[i, 'pres_2018_tot'] = '-'
            df.loc[i, 'goal_2018_tot'] = '-'
            df.loc[i, 'assist_2018_tot'] = '-'

        # all 2017 =============================================================

        trs = page17.select('div#yw2 tfoot')[0].find_all('tr')        
        for tr in trs:
            tds = tr.find_all('td')
            pres = tds[2].text.strip()
            gol = tds[3].text.strip()
            assist = tds[4].text.strip()
        df.loc[i, 'pres_2017_tot'] = pres
        df.loc[i, 'goal_2017_tot'] = gol
        df.loc[i, 'assist_2017_tot'] = assist

        # all tot =============================================================

        trs = pagetot.select('div#yw2 tfoot')[0].find_all('tr')        
        for tr in trs:
            tds = tr.find_all('td')
            pres = tds[2].text.strip()
            gol = tds[3].text.strip()
            assist = tds[4].text.strip()
        df.loc[i, 'pres_tot'] = pres
        df.loc[i, 'goal_tot'] = gol
        df.loc[i, 'assist_tot'] = assist

        # champions 2018 ========================================================
        
        try:
            trs = page18.select('div#yw2 tbody')[0].find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                pres = '-'
                gol = '-'
                assist = '-'
                if tds[1].text.strip()=='Champions League':
                    pres = tds[2].text.strip()
                    gol = tds[3].text.strip()
                    assist = tds[4].text.strip()
                    break
            df.loc[i, 'pres_2018_CL'] = pres
            df.loc[i, 'goal_2018_CL'] = gol
            df.loc[i, 'assist_2018_CL'] = assist
        except:
            df.loc[i, 'pres_2018_CL'] = '-'
            df.loc[i, 'goal_2018_CL'] = '-'
            df.loc[i, 'assist_2018_CL'] = '-'

        # champions 2017 ========================================================

        trs = page17.select('div#yw2 tbody')[0].find_all('tr')
        for tr in trs:
            tds = tr.find_all('td')
            pres = '-'
            gol = '-'
            assist = '-'
            if tds[1].text.strip()=='Champions League':
                pres = tds[2].text.strip()
                gol = tds[3].text.strip()
                assist = tds[4].text.strip()
                break
        df.loc[i, 'pres_2017_CL'] = pres
        df.loc[i, 'goal_2017_CL'] = gol
        df.loc[i, 'assist_2017_CL'] = assist

        # champions tot  ========================================================

        trs = pagetot.select('div#yw2 tbody')[0].find_all('tr')
        for tr in trs:
            tds = tr.find_all('td')
            pres = '-'
            gol = '-'
            assist = '-'
            if tds[1].text.strip()=='Champions League':
                pres = tds[2].text.strip()
                gol = tds[3].text.strip()
                assist = tds[4].text.strip()
                break
        df.loc[i, 'pres_tot_CL'] = pres
        df.loc[i, 'goal_tot_CL'] = gol
        df.loc[i, 'assist_tot_CL'] = assist
        
        print((i,df_urls.nome[i]), end=' ')

(298, 'Danilo Pereira') (299, 'José Callejón') 

In [67]:
df.to_csv('columns/performances.csv', index=False)

# 14 - Ultimi newspaper

In [ ]:
df = read_csv('tables/nome_nomechiave.csv')
df['nome_url'] = [x.replace(" ","+").replace("-","+").lower() for x in df.nome_chiave]

## onzemondial.com

In [21]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:www.gazzetta.it&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['onzemondial_google'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.onzemondial_google[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'onzemondial_google'] = result
        print((i, df.nome[i]), end=' ')

In [ ]:
l = [x.replace('About ', '').replace(' results', '').replace(',', '').replace(' result','') for x in df.onzemondial_google]
df1 = pd.DataFrame({'onzemondial_google': l})

## thesun

In [33]:
df['thesun'] = np.nan

In [34]:
s1 = "https://www.thesun.co.uk/?s="

lista_2 = [s1+x for x in df.nome_url]

In [ ]:
for i in range(300):
    if isinstance(df.thesun[i], float):
        url = lista_2[i]
        page = BS(get(url).text,'lxml')
        n_pages = page.select("div.search-result-count p")[0].text.strip().split(" ")[0].replace(",","")
        df.loc[i, 'thesun'] = n_pages
        print((i, df.nome[i]), end=' ')

In [46]:
df1['thesun'] = df.thesun

## marca.com

In [37]:
df['marca'] = np.nan

In [38]:
s1 = "https://cgi.marca.com/buscador/archivo_marca.html?q="

lista_3 = [s1+unidecode(x) for x in df.nome_url]

In [44]:
df.loc[256, 'marca'] = '0'

In [ ]:
driver = webdriver.Chrome(executable_path='driver/chromedriver')
driver.maximize_window()

results = []

for i in range(300):
    if isinstance(df.marca[i], float):
        url = lista_3[i]
        driver.get(url)
        sleep(np.random.uniform(2))
        
        result = driver.find_element_by_class_name('porsecciones').find_element_by_tag_name('a').text
        result = re.findall(r'\d+', result)
        df.loc[i, 'marca'] = result
        print((i,df.nome[i]), end=' ')
    
driver.quit()
df1['marca'] = [x[0] for x in df.marca]

## bild.de

In [56]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:bild.de&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['bild_google'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.bild_google[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'bild_google'] = result
        print((i, df.nome[i]), end=' ')

In [ ]:
l = [x.replace('Ungefähr ', '').replace(' Ergebnisse', '').replace(' Ergebnis', '').replace('.', '') for x in df.bild_google]
df1['bild.de'] = l

## eurosport.de

In [66]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:eurosport.de&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['eurosportDE_google'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.eurosportDE_google[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'eurosportDE_google'] = result
        print((i, df.nome[i]), end=' ')

In [ ]:
l = [x.replace('Ungefähr ', '').replace(' Ergebnisse', '').replace(' Ergebnis', '').replace('.', '') for x in df.eurosportDE_google]
df1['eurosportDE_google'] = l

## lequipe

In [73]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:lequipe.fr&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['lequipe_google'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.lequipe_google[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'lequipe_google'] = result
        print((i, df.nome[i]), end=' ')

In [ ]:
l = [''.join(re.findall('(\d)',unidecode(x))) for x in df.lequipe_google]
df1['lequipe_google'] = l

## mundodeportivo

In [87]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:mundodeportivo.com&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['mundodeportivo_google'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.mundodeportivo_google[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'mundodeportivo_google'] = result
        print((i, df.nome[i]), end=' ')

In [ ]:
l = [''.join(re.findall('(\d)',unidecode(x))) for x in df.mundodeportivo_google]
df1['mundodeportivo_google'] = l

## espn

In [94]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:espn.com&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['espn_google'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.espn_google[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'espn_google'] = result
        print((i, df.nome[i]), end=' ')

In [ ]:
l = [''.join(re.findall('(\d)',unidecode(x))) for x in df.espn_google]
df1['espn_google'] = l

## beinsports

In [100]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:beinsports.com&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['bein_google'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.bein_google[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'bein_google'] = result
        print((i, df.nome[i]), end=' ')

In [ ]:
l = [''.join(re.findall('(\d)',unidecode(x))) for x in df.bein_google]
df1['bein_google'] = l

## eurosport.fr

In [5]:
url = 'https://docs.google.com/spreadsheets/d/1RPiYmNxy8NWKp-uu_swOkRkTuq9cqJNnzJpl5BPliDM/export?format=csv&id=1RPiYmNxy8NWKp-uu_swOkRkTuq9cqJNnzJpl5BPliDM&gid=2001946666'
r = get(url)
data = r.content.decode('utf-8')
df = pd.read_csv(StringIO(data))
df = pd.DataFrame(df[['nome_chiave']])
df['nome_url'] = [x.replace(" ","+").replace("-","+").lower() for x in df.nome_chiave]

In [6]:
s1 = 'https://www.google.com/search?q='
s2 = '+site:eurosport.fr&source=lnms&tbm=nws'
lista_1 = [s1 + x + s2 for x in df.nome_url]
df['eurosportFR'] = np.nan

In [ ]:
for i in range(300):
    if isinstance(df.eurosportFR[i], float):
        url = lista_1[i]
        result = BS(get(url).text, 'lxml').select('#resultStats')[0].text
        df.loc[i, 'eurosportFR'] = result
        if result == '':
            df.loc[i, 'eurosportFR'] = np.nan
        print((i, df.nome_chiave[i], result), end=' ')

In [11]:
l = [''.join(re.findall('(\d)',unidecode(x))) for x in df.eurosportFR]
df['eurosportFR'] = l
df.eurosportFR[df['eurosportFR'] == ''] = '1'
df[['eurosportFR']].to_csv('columns/eurosportFR.csv', index=False)